# Challenge 3 Solution!

This notebook contains the solution to the Chapter 3 Notebook called

`challenge_post-requests.ipynb`

### Your Challenge

Create the function that calls the `/Account/v1/Authorized` endpoint which is used to check if a given user is authorized or not.

**STEPS:**

1. Create an authorized user
2. Assert they are authorized using this endpoint
3. Then delete the user.

**OPTIONAL BONUS:**

Define Models with Pydantic to use for the `create_user()` and `generate_token()` functions!

In [ ]:
""" Install pydantic (only if needed)

* Don't run this cell if you already have pydantic installed in your project
* Run this cell if you are running this notebook in Google Colaboratory
"""
!pip install pydantic

In [1]:
""" Define our models """
from typing import List
from pydantic import BaseModel


class BookModel(BaseModel):
    isbn: str
    title: str
    subTitle: str
    author: str
    publish_date: str  # they combine camelCase and snake_case... this is a design flaw and should stick with a single convention
    publisher: str
    pages: int
    description: str
    website: str


class UserModel(BaseModel):
    userID: str  # their docs show `userId`, but their implementation has `userID`... a breach of contract and a bug!
    username: str
    books: List[BookModel]


class TokenModel(BaseModel):
    token: str
    expires: str  # this Date object is in string form, but Pydantic can validate Dates as well. Check out their docs for more info!
    status: str
    result: str

In [2]:
""" Define our functions using parameters and our models as a return type """
import requests
from requests import Response


BASE_URL = 'https://demoqa.com'
ACCOUNT_URL = BASE_URL + '/Account/v1'


def create_user(username, password) -> UserModel:
    payload = {'userName': username, 'password': password}
    response = requests.post(f'{ACCOUNT_URL}/User', json=payload)
    if response.ok:
        return UserModel(**response.json())
    else:
        # if response is not ok, raise an error. Maybe a ValueError, ConnectionError, or your own custom error!
        raise ValueError(f'Unable to create user: {response.content}')


def generate_token(username, password) -> TokenModel:
    payload = {'userName': username, 'password': password}
    response = requests.post(f'{ACCOUNT_URL}/GenerateToken', json=payload)
    if response.ok:
        return TokenModel(**response.json())
    else:
        raise ValueError(f'Unable to generate token: {response.content}')


def delete_user(user_id, token) -> Response:
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.delete(f'{ACCOUNT_URL}/User/{user_id}', headers=headers)
    if response.ok:
        return response
    else:
        raise ValueError(f'Unable to delete user: {response.content}')


def is_authorized(username, password) -> bool:
    payload = {'userName': username, 'password': password}
    response = requests.post(f'{ACCOUNT_URL}/Authorized', json=payload)

    # we can also write our if/else block this way
    if not response.ok:
        raise ConnectionError(f'Unable to authorize user: {response.content}')

    return response.json()

In [3]:
""" Create a user, generate a token for that user, then assert that they're authorized """
CREDENTIALS = 'post requests username', 'P@$$w0rd'

user  = create_user(*CREDENTIALS)
token = generate_token(*CREDENTIALS)

assert is_authorized(*CREDENTIALS) is True

deletion = delete_user(user.userID, token.token)
print('User deleted?', deletion.ok)

User deleted? True
